In [2]:
#Загрузка файла в формате .json
import json

""" Каждый словарь состоит из следующих элементов:
id - номер вакансии
title - название вакансии
desc - описание вакансии
values - список словарей со следующими ключами: start - индекс начала скилла, end - индекс конца скилла, skill - сущность
"""

with open('train.json', encoding='UTF8') as file:
    data = json.load(file)

In [3]:
#выгрузка файла в формате .json
import json

with open('train.json', 'w', encoding='UTF8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [43]:
#создание таблицы (при необходимости)
import pandas as pd
import numpy as np
df = pd.DataFrame.from_dict(data)

In [5]:
df

,id,title,desc,values
0,868,Птицевод,"АО «ДРУЖБА НАРОДОВ НОВА», входящее в состав кр...","[{'start': 119, 'end': 147, 'skill': 'Выращива..."
1,1042,Бульдозерист,Требования: Образование - от 9 классов и выше...,"[{'start': 137, 'end': 158, 'skill': 'Физическ..."
2,4390,Специалист по качеству,Обязанности: разработка технической документа...,"[{'start': 16, 'end': 90, 'skill': 'разработка..."
3,4678,Слесарь-сборщик,На производственное предприятие требуются Слес...,"[{'start': 94, 'end': 197, 'skill': 'Сборка от..."
4,5228,Мастер по наращиванию волос,Обязанности: Студия по наращиванию волос пpиг...,[{'skill': 'cтpемлениe оттачивать полученные з...
...,...,...,...,...
3996,45,"Главный специалист по отоплению, вентиляции и ...",Обязанности: Проектирование в области гражданс...,"[{'start': 15, 'end': 82, 'skill': 'Проектиров..."
3997,1768,Менеджер по продажам со знанием английского языка,Присоединяйтесь к крупнейшей в Европе онлайн-ш...,"[{'start': 183, 'end': 243, 'skill': 'Проводит..."
3998,2932,Руководитель направления лизингового кредитования,В нашей команде открыта позиция «Руководитель ...,"[{'start': 211, 'end': 255, 'skill': 'выстроит..."
3999,2577,Бухгалтер,Обязанности Ведение участка по основным средст...,"[{'start': 14, 'end': 74, 'skill': 'Ведение уч..."


<b> Создание векторов для каждого скилла для оценивания модели (при необходимости) </b>

In [6]:
!pip install -U sentence-transformers

Для создания векторов рекомендуем использовать любую модель по ссылке: https://github.com/avidale/encodechka

In [7]:
from sentence_transformers import SentenceTransformer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("BAAI/bge-m3", device=device)
#При возникновении TypeError рекомендуем заново запустить верхнюю строчку

C:\Users\shche\.conda\envs\ds\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\Users\shche\.conda\envs\ds\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [10]:
df.head()

,id,title,desc,values
0,868,Птицевод,"АО «ДРУЖБА НАРОДОВ НОВА», входящее в состав кр...","[{'start': 119, 'end': 147, 'skill': 'Выращива..."
1,1042,Бульдозерист,Требования: Образование - от 9 классов и выше...,"[{'start': 137, 'end': 158, 'skill': 'Физическ..."
2,4390,Специалист по качеству,Обязанности: разработка технической документа...,"[{'start': 16, 'end': 90, 'skill': 'разработка..."
3,4678,Слесарь-сборщик,На производственное предприятие требуются Слес...,"[{'start': 94, 'end': 197, 'skill': 'Сборка от..."
4,5228,Мастер по наращиванию волос,Обязанности: Студия по наращиванию волос пpиг...,[{'skill': 'cтpемлениe оттачивать полученные з...


Находим все скиллы и создаём массив массивов скиллов для каждой вакансии

In [26]:
mas_y = []
for i in range(df.shape[0]):
    all_skills = []
    for j in range(len(df.values[i][3])):
            all_skills.append(df.values[i][3][j]['skill'])
    mas_y.append(all_skills)

In [27]:
len(mas_y)

4001

Затем, будем разрезать описания вакансий на токены и будем проверять что из них является скиллом

In [38]:
import re

def split_on_multiple_delimiters(s, delimiters):
    """Разделяем строку по нескольким разделителям."""
    regex_pattern = '[' + ''.join(re.escape(d) for d in delimiters) + ']'
    return re.split(regex_pattern, s)
    
def tokenizer(word):
    return split_on_multiple_delimiters(word, [',', '.', ';', ':'])

mas_x = []
for i in range(df.shape[0]):
    mas_x.append(tokenizer(df['desc'][i]))    
for i in range(len(mas_x)):
    for j in range(len(mas_x[i])):
        mas_x[i][j] = mas_x[i][j].strip()

In [39]:
mas_x

[['АО «ДРУЖБА НАРОДОВ НОВА»',
  'входящее в состав крупного агропромышленного холдинга приглашаем птицевода   Обязанности',
  'Выращивание цыплят-бройлеров',
  'Поддержание оптимального санитарно-ветеринарного состояния на производственном участке по выращиванию птицы',
  'Контроль работы оборудования',
  'Уход за птицепоголовьем',
  'Сбор павшей птицы',
  'Подготовка птичников к приемке новой партии цыплят',
  'Подготовка птицы к сдаче на убой',
  'Выполнения всех технологических мероприятий связанных с выращиванием птицы',
  'Соблюдение культуры производства',
  'Соблюдение правил внутреннего трудового распорядка',
  'Требования',
  'Знание правил ухода и содержания сельскохозяйственной птицы',
  'Основы трудового законодательства Российской Федерации',
  'Правила и нормы охраны труда',
  'Условия',
  'Сменный график работы (дневная 8',
  '00 до 17',
  '00',
  'ночная с 17',
  '00 до 8',
  '00) Оформление по ТК РФ',
  'официальная заработная плата',
  'Работа на стабильном',
  'разви

In [70]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
model.encode(mas_x[0][0]).shape

(384,)

Теперь, построим массив массивов y. Он будет содержать True если токен является навыком и False если нет

In [44]:
y = []
for i in range(len(mas_x)):
    for j in range(len(mas_x[i])):
        y.append(int(mas_x[i][j] in mas_y[i]))
y = np.array(y)

И превратим все токены в векторы

In [71]:
x = []
for i in range(len(mas_x)):
    print((i/len(mas_x))*100)
    for j in range(len(mas_x[i])):
        x.append(model.encode(mas_x[i][j]))

0.0
0.024993751562109475
0.04998750312421895
0.07498125468632841
0.0999750062484379
0.12496875781054737
0.14996250937265682
0.17495626093476632
0.1999500124968758
0.22494376405898525
0.24993751562109473
0.2749312671832042
0.29992501874531363
0.3249187703074231
0.34991252186953264
0.3749062734316421
0.3999000249937516
0.42489377655586097
0.4498875281179705
0.47488127968008
0.49987503124218946
0.5248687828042989
0.5498625343664084
0.5748562859285179
0.5998500374906273
0.6248437890527367
0.6498375406148462
0.6748312921769558
0.6998250437390653
0.7248187953011748
0.7498125468632842
0.7748062984253936
0.7998000499875032
0.8247938015496127
0.8497875531117219
0.8747813046738315
0.899775056235941
0.9247688077980505
0.94976255936016
0.9747563109222694
0.9997500624843789
1.0247438140464884
1.0497375656085979
1.0747313171707074
1.0997250687328168
1.1247188202949263
1.1497125718570358
1.1747063234191453
1.1997000749812545
1.224693826543364
1.2496875781054735
1.274681329667583
1.2996750812296924
1.

In [74]:
x = np.array(x)

In [75]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import numpy as np

# Разделение данных на обучающую и проверочную выборки
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Создание модели
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=384))
model.add(Dense(1, activation='sigmoid'))

# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), verbose=1)

# Проверка точности модели
score = model.evaluate(x_test, y_test, verbose=0)
print('Тестовый набор точности:', score[1])

Epoch 1/10
2652/2652 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8650 - loss: 0.3150 - val_accuracy: 0.8841 - val_loss: 0.2582
Epoch 2/10
2652/2652 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8856 - loss: 0.2535 - val_accuracy: 0.8883 - val_loss: 0.2457
Epoch 3/10
2652/2652 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8940 - loss: 0.2344 - val_accuracy: 0.8906 - val_loss: 0.2387
Epoch 4/10
2652/2652 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8960 - loss: 0.2300 - val_accuracy: 0.8924 - val_loss: 0.2368
Epoch 5/10
2652/2652 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8995 - loss: 0.2219 - val_accuracy: 0.8903 - val_loss: 0.2375
Epoch 6/10
2652/2652 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9022 - loss: 0.2154 - val_accuracy: 0.8934 - val_loss: 0.2352
Epoch 7/10
2652/2652 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9044 - loss: 0.2093 - val_accuracy: 0.8907 - val_loss: 0.2403
Epoch 8/10
2652/2652 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9047 - loss: 0.2070 - 

In [78]:
for i in model.predict(x_test):
    print(i)

663/663 ━━━━━━━━━━━━━━━━━━━━ 1s 879us/step
[0.0237123]
[0.5128841]
[0.6815841]
[0.00282212]
[0.19195844]
[0.00059017]
[0.6093025]
[0.00052488]
[0.76223147]
[0.09434381]
[0.3736252]
[0.02942448]
[0.01299639]
[0.03023807]
[0.00072965]
[0.07764713]
[0.00072965]
[0.00028367]
[0.1098479]
[0.00041124]
[0.00342833]
[0.01051303]
[0.18358448]
[0.00042837]
[0.78987265]
[0.03863794]
[2.9556202e-06]
[0.00078022]
[0.00089942]
[2.5979323e-06]
[0.00078022]
[0.34714553]
[0.01589695]
[0.67725074]
[0.27221027]
[0.00402444]
[0.01204251]
[0.20805228]
[0.00343666]
[5.2068866e-05]
[9.2993145e-07]
[6.523362e-06]
[0.03697678]
[0.00051484]
[0.40725228]
[0.00095583]
[0.00066676]
[0.02299222]
[0.39651853]
[0.0031309]
[0.00181586]
[0.5949577]
[0.0275009]
[0.00066676]
[0.11933966]
[0.39647365]
[0.7093655]
[0.0018209]
[6.789474e-06]
[0.00034084]
[0.76052004]
[0.0408696]
[0.2090957]
[0.03345957]
[0.67328477]
[0.00213493]
[0.00194671]
[0.02477122]
[0.4811447]
[0.00899159]
[0.8657389]
[4.1674116e-06]
[0.20101684]
[0.1

Модель обучена. Визуальная проверка, что модель принимает за скиллы

In [86]:
new_mas_x = []
for i in mas_x:
    for j in i:
        new_mas_x.append(j)

test_mas = x[:100]
res = model.predict(test_mas)
for i in range(len(res)):
    if res[i] > 0.5:
        print(new_mas_x[i])

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Выращивание цыплят-бройлеров
Контроль работы оборудования
Уход за птицепоголовьем
Сбор павшей птицы
Подготовка птичников к приемке новой партии цыплят
Подготовка птицы к сдаче на убой
Выполнения всех технологических мероприятий связанных с выращиванием птицы
Соблюдение культуры производства
Соблюдение правил внутреннего трудового распорядка
Знание правил ухода и содержания сельскохозяйственной птицы
Навыки выполнения прихваток с использованием ручной и полуавтоматической сварки
аккуратность


Код для запуска:


In [ ]:
def find_skills(main_string, substrings):
    skills = []
    for skill in substrings:
        try:
            start = main_string.index(skill)
            end = start + len(skill)
            skills.append({'start': start, 'end': end, 'skill':skill})
        except ValueError:
            pass
    return skills

with open('test.json', encoding='UTF8') as file:
    data = json.load(file)

df = pd.DataFrame.from_dict(data)

import re

def split_on_multiple_delimiters(s, delimiters):
    """Разделяем строку по нескольким разделителям."""
    regex_pattern = '[' + ''.join(re.escape(d) for d in delimiters) + ']'
    return re.split(regex_pattern, s)
    
def tokenizer(word):
    return split_on_multiple_delimiters(word, [',', '.', ';', ':'])

mas_x = []
for i in range(df.shape[0]):
    mas_x.append(tokenizer(df['desc'][i]))
for i in range(len(mas_x)):
    for j in range(len(mas_x[i])):
        mas_x[i][j] = mas_x[i][j].strip()


final = []
for i in range(len(mas_x)):
    x = []
    for j in range(len(mas_x[i])):
        x.append(model.encode(mas_x[i][j]))
        test_mas = x
        res = model.predict(test_mas)
        ans = []
        for i in range(len(res)):
            if res[i] > 0.5:
                ans.append(new_mas_x[i])
        final.append(find_skills(df['desc'][i], ans))
final = np.array(final)
df['values'] = final
data = df

In [48]:
#Data - список словарей
from tqdm import tqdm

for d in tqdm(data):
    for value in d['values']:
        value['emb'] = list(map(str, model.encode(value['skill']))) #строим эмбеддинг для каждого скилла и превращаем в строку (для сохранения в JSON)

  0%|          | 2/1471 [00:04<51:24,  2.10s/it]


KeyboardInterrupt: 

In [46]:
""" Переделываем словарь в следующий формат:
id - номер вакансии
desc - описание вакансии
skills - список скиллов
embs - список эмбеддингов 
(При необходимости)"""

skill_list = []
for d in data:
    vac_dict = {'id':d['id'], 'desc': d['desc']}
    vac_dict['skills'] = []
    vac_dict['embs'] = []
    for sk in d['values']:
        vac_dict['skills'].append(sk['skill'])
        vac_dict['embs'].append(sk['emb'])
    skill_list.append(vac_dict)

In [49]:
#сохраняем в json
import json

with open('test_emb.json', 'w', encoding='UTF8') as f:
    json.dump(skill_list, f, ensure_ascii=False, indent=4)